# D3

In [ ]:
%%html

<script type="text/javascript" src="https://d3js.org/d3.v3.min.js"></script>
<script type="text/javascript" src="https://tvinci.github.io/webs/js/dc.js"></script>
<script type="text/javascript" src="https://tvinci.github.io/webs/js/crossfilter.js"></script>
<link rel="stylesheet" type="text/css" href="https://tvinci.github.io/webs/css/dc.css">
    
<div id="chart-ring-year"></div>
<div id="chart-line-hitsperday"></div>

<script>
var hitslineChart  = dc.lineChart("#chart-line-hitsperday");
var data = [
	{date: "12/27/2012", http_404: 2, http_200: 190, http_302: 100},
	{date: "12/28/2012", http_404: 2, http_200: 10, http_302: 100},
	{date: "12/29/2012", http_404: 1, http_200: 300, http_302: 200},
	{date: "12/30/2012", http_404: 2, http_200: 90, http_302: 0},
	{date: "12/31/2012", http_404: 2, http_200: 90, http_302: 0},
	{date: "01/06/2013", http_404: 2, http_200: 200, http_302: 1},
	{date: "01/07/2013", http_404: 1, http_200: 200, http_302: 100}
];
var ndx = crossfilter(data);
var parseDate = d3.time.format("%m/%d/%Y").parse;
data.forEach(function(d) {
	d.date = parseDate(d.date);
  d.total= d.http_404+d.http_200+d.http_302;
  d.Year=d.date.getFullYear();
});
var dateDim = ndx.dimension(function(d) {return d.date;});
var hits = dateDim.group().reduceSum(function(d) {return d.total;});
var minDate = dateDim.bottom(1)[0].date;
var maxDate = dateDim.top(1)[0].date;
hitslineChart.width(500).height(200).dimension(dateDim).group(hits).x(d3.time.scale()
    .domain([minDate,maxDate])).brushOn(false).yAxisLabel("Hits per day");
var yearRingChart   = dc.pieChart("#chart-ring-year");
var yearDim  = ndx.dimension(function(d) {return +d.Year;});
var year_total = yearDim.group().reduceSum(function(d) {return d.http_200+d.http_302;});
yearRingChart.width(150).height(150).dimension(yearDim).group(year_total).innerRadius(30);
dc.renderAll();
</script>

In [ ]:
%%html
<body>
  <button id='add'>add</button>
  <button id='clear'>clear</button>
  <button id='display'>display</button>
</body>
<script>
// https://googlechrome.github.io/samples/idb-getall/
// https://developer.mozilla.org/en-US/docs/Web/API/IndexedDB_API/Browser_storage_limits_and_eviction_criteria#Storage_limits
// https://developers.google.com/web/ilt/pwa/working-with-indexeddb
var addButton = document.querySelector('#add');
var displayButton = document.querySelector('#display');
var clearButton = document.querySelector('#clear');

var DB_NAME = 'timestamps';
var DB_VERSION = 1;
var STORE_NAME = 'store';
var db;

var request = indexedDB.open(DB_NAME, DB_VERSION);
request.onupgradeneeded = () => {
  console.log('Upgrading Store.');
  if (!db.objectStoreNames.contains(STORE_NAME)) {
    request.result.createObjectStore(STORE_NAME, {autoIncrement: true});
  }
};
request.onsuccess = () => { db = request.result; };
clearButton.addEventListener('click', () => {
  var transaction = db.transaction(STORE_NAME, 'readwrite');
  var objectStore = transaction.objectStore(STORE_NAME);
  objectStore.clear().onsuccess = () => {
    console.log('Cleared timestamps from IndexedDB.');
  };
});
addButton.addEventListener('click', () => {
  var transaction = db.transaction(STORE_NAME, 'readwrite');
  var objectStore = transaction.objectStore(STORE_NAME);
  objectStore.put(Date.now()).onsuccess = () => {
    console.log('Added timestamp to IndexedDB.');
  };
});
displayButton.addEventListener('click', () => {
  var transaction = db.transaction(STORE_NAME, 'readonly');
  var objectStore = transaction.objectStore(STORE_NAME);
  if ('getAll' in objectStore) {
    objectStore.getAll().onsuccess = event => {
      let timestamps = event.target.result
      console.log('There are ' + timestamps.length + ' timestamp(s) saved in IndexedDB: ' + timestamps.join(', ') );
    };
  } 
});
</script>